In [84]:
import pandas as pd
import numpy as np
import urllib.error as err
import urllib.request as libreq
import urllib
import time, socket, sys, os
from bs4 import BeautifulSoup as bs

## READ IN CSV

In [85]:
player = []
team = []

for dire in os.listdir(os.getcwd()):
    if "phs_" in dire:
        cur = os.path.join(os.getcwd(), dire, "")
        for file in os.listdir(cur):
            try:
                df = pd.read_csv(cur + file)[['player', 'team']].to_numpy()
                player += set(df[:, 0].tolist())
                team   += set(df[:, 1].tolist())
            except:
                df = pd.read_csv(cur + file)[['player_name', 'team_name']].to_numpy()
                player += set(df[:, 0].tolist())
                team   += set(df[:, 1].tolist())


player = set(player)
team   = set(team)
player = sorted(player)
team   = sorted(team)
#player, team

In [86]:
team

['Atlanta Reign',
 'Boston Uprising',
 'Chengdu Hunters',
 'Dallas Fuel',
 'Florida Mayhem',
 'Guangzhou Charge',
 'Hangzhou Spark',
 'Houston Outlaws',
 'London Spitfire',
 'Los Angeles Gladiators',
 'Los Angeles Valiant',
 'New York Excelsior',
 'Paris Eternal',
 'Philadelphia Fusion',
 'San Francisco Shock',
 'Seoul Dynasty',
 'Shanghai Dragons',
 'Toronto Defiant',
 'Vancouver Titans',
 'Washington Justice']

## Clean up player name in url

In [87]:
for pl in range(len(player)):
    if player[pl].lower() == 'although':
        player[pl] = 'ALTHOUGH'
    elif player[pl].lower() == 'bebe':
        player[pl] = 'BeBe'
    elif player[pl].lower() == 'bumper':
        player[pl] = 'Bumper'
    elif player[pl].lower() == 'choihyobin':
        player[pl] = 'ChoiHyoBin'
    elif player[pl].lower() == 'colourhex':
        player[pl] = 'Colourhex'
    elif player[pl].lower() == 'dreamer':
        player[pl] = 'Dreamer'
    elif player[pl].lower() == 'fdgod':
        player[pl] = 'FDGoD'
    elif player[pl].lower() == 'fearless':
        player[pl] = 'Fearless'
    elif player[pl].lower() == 'fits':
        player[pl] = 'FITS'
    elif player[pl].lower() == 'gargoyle':
        player[pl] = 'Gargoyle'
    elif player[pl].lower() == 'garry':
        player[pl] = 'Garry'
    elif player[pl].lower() == 'gangnamjin':
        player[pl] = 'Gangnamjin'
    elif player[pl].lower() == 'izayaki':
        player[pl] = 'Izayaki'
    elif player[pl].lower() == 'jiqiren':
        player[pl] = 'jiqiren'
    elif player[pl].lower() == 'jake':
        player[pl] = 'JAKE'
    elif player[pl].lower() == 'jmac':
        player[pl] = 'JMAC'   
    elif player[pl].lower() == 'leejaegon':
        player[pl] = 'LeeJaeGon'
    elif player[pl].lower() == 'mcgravy':
        player[pl] = 'McGravy'
    elif player[pl].lower() == 'onlywish':
        player[pl] = 'OnlyWish'
    elif player[pl].lower() == 'quatermain':
        player[pl] = 'Quatermain'
    elif player[pl].lower() == 'saebyeolbe':
        player[pl] = 'Saebyeolbe'
    elif player[pl].lower() == 'sp9rk1e':
        player[pl] = 'Sp9rk1e'
    elif player[pl].lower() == 'swon':
        player[pl] = 'Swon'
    elif player[pl].lower() == 'spree':
        player[pl] = 'SPREE'
    elif player[pl].lower() == 'toyou':
        player[pl] = 'Toyou'
    elif player[pl].lower() == 'ttuba':
        player[pl] = 'TTuba'
    elif player[pl].lower() == 'whoru':
        player[pl] = 'WhoRU'
    elif player[pl].lower() == 'zunba':
        player[pl] = 'zunba'
    elif player[pl].lower() == 'blasé' or player[pl].lower() == 'blase':
        player[pl] = 'Blasé'
    elif player[pl].lower() == 'bqb':
        player[pl] = 'BQB'
    elif player[pl].lower() == 'snow':
        player[pl] = 'Snow_(Brazilian_player)'
        player.append('Snow_(Mikias_Yohannes)')
for tm in range(len(team)):
    if team[tm].lower() == 'chengdu hunters':
        team[tm] = 'Chengdu_Hunters'

## "global" var before running the acutal soup part

In [88]:
base_url = 'https://liquipedia.net/overwatch/'
list_player = []
already_dealt = []
already_dealt_team = []
error_list = []
history_list = []
list_team = []
cols = ['Romanized Name',
  'Birth','Country','Status','Team','Approx. Total Earnings','Common Team Role(s)','Signature Hero','Player Name']

## Actual web crawling

In [89]:
for p in player:
    if p.lower() in already_dealt:
        continue
    print(p)
    
    url = base_url + p

    attempts, success = 0, 0
    while attempts < 2 and not success:
        try:
            with libreq.urlopen(url, timeout=30) as page:  # request info
                doc = page.read().decode('utf-8')
            success = 1
        except (err.URLError, err.HTTPError) as e:  # catch Internet error
            attempts += 1
            print(f"\n{e.reason}-{p}\n")
            error_list.append(p)
            if e.reason == 'NOT FOUND':  # return e if 404: page not found
                print("page not found.")
            time.sleep(1)
        except socket.timeout:  # catch timeout error
            attempts += 1
            print("\nTimeout error\n")
        except UnicodeEncodeError as e:
            with libreq.urlopen("http:" + urllib.parse.quote(url.split(':')[1])) as page:
                doc = page.read()
            success = 1
            print("Not Recognized Characters.")
        except:  # catch all rest error
            attempts += 1
            print("\nUnknown error\n")
        
    if not success:
        continue


    soup = bs(doc)
    t = soup.findAll("div", {"class":'infobox-cell-2'})
    a = soup.findAll("div", {"class":'infobox-center'})
    mylist = []
    for i in range(len(a)):
        te = a[i].text.replace("–", "-").replace("—", "-").replace('\xa0', ' ')  
        if '-' in te:
            mylist.append(te)
            

    t2 = ''.join(mylist).strip().split('\n')
    l = {}
    

    for idx in range(0, len(t), 2):
        s = t[idx].text
        try:
            if s[:-1] not in cols:
                continue
            elif s == 'Birth:':
                l[s[:-1]] = t[idx+1].text.split('(')[1].replace(')', '')
            elif s == 'Common Team Role(s):':
                print('*'*20)
                print(t[idx+1].text)
                print('*'*20)
                l[s[:-1]] = t[idx+1].text.replace(',', ';').replace('\n', '; ').replace(' ', '; ')
            elif s == 'Signature Hero:':
                l[s[:-1]] = t[idx+1].text.replace('\n', ';').replace(':', '').strip().replace(' ', '; ').replace(", ", "; ").replace(";;", ";")
            elif s == 'Approx. Total Earnings:':
                l[s[:-1]] = t[idx+1].text.replace(',', '')
            else:
                l[s[:-1]] = t[idx+1].text.replace(",", ";")
        except:
            l[s[:-1]] = t[idx+1].text
            
        #deal with history part

    for idx in range(0, len(t2), 1):
        try:
            l2 = {}
            t2[idx] = t2[idx].replace("–", "-")
            l2['Player Name'] = p
            l2["start_time"] = t2[idx].split(' - ')[0]
            l2["end_time"] = t2[idx].split(' - ')[1].split('  ')[0]
            l2["Team"] = t2[idx].split(' - ')[1].split('  ')[1]
            history_list.append(l2)
        except:
            print("ha"*20)
            
        

    l['Player Name'] = p 
    list_player.append(l)
    already_dealt.append(p.lower())


ANS
********************
DPS
********************
ATing
********************
Tank
********************
Ado
********************
DPS
********************
Adora
********************
DPS
********************
Agilities
********************
DPS
********************
Aid
********************
CoachSupport
********************
AimGod
********************
Support
********************
Alarm
********************
Support
********************
Altering
********************
Coach
********************
ALTHOUGH
********************
DPS
********************
Anamo
********************
Support
********************
Apply
********************
DPS
********************
ArHaN
********************
CoachDPS
********************
ArK
********************
Support
********************
Architect
********************
DPS
********************
Asher
********************
DPS
********************
BeBe
********************
Support
********************
BERNAR
********************
Tank
********************
Bumper
************

********************
Support
********************
Leave
********************
DPS
********************
LeeJaeGon
********************
Support
********************
Lengsa
********************
Support
********************
LhCloudy
********************
Tank
********************
LiNkzr
********************
DPS
********************
Libero
********************
DPS
********************
Logix
********************
DPS
********************
Lr1s
********************
Support
********************
Luffy
********************
Support
********************
M1ka
********************
Support
********************
MG
********************
Tank
********************
Mandu
********************
Support
********************
Mangachu
********************
DPSCoach
********************
Manneten
********************
Tank
********************
Mano
********************
Tank
********************
Marve1
********************
Tank
********************
Masaa
********************
Support
********************
McGravy
*********

********************
Tank
********************
ryujehong
********************
Support
********************
sHockWave
********************
DPS
********************
sayaplayer
********************
DPS
********************
sharyk
********************
Tank
********************
shu
********************
Support
********************
silkthread
********************
DPS
********************
sinatraa
********************
DPS
********************
sleepy
********************
Support
********************
smurf
********************
Tank
********************
snillo
********************
DPS
********************
super
********************
Tank
********************
ta1yo
********************
DPS
********************
tobi
********************
Support
********************
uNKOE
********************
Support
********************
xQc
********************
Tank
********************
xepheR
********************
Tank
********************
zYKK
********************
DPS
********************
zappis
******************

In [90]:
for t in team:
    if t.lower() in already_dealt_team:
        continue
    print(t)
    
    url = base_url + t

    attempts, success = 0, 0
    while attempts < 2 and not success:
        try:
            with libreq.urlopen(url, timeout=30) as page:  # request info
                doc = page.read().decode('utf-8')
            success = 1
        except (err.URLError, err.HTTPError) as e:  # catch Internet error
            attempts += 1
            print(f"\n{e.reason}-{p}\n")
            error_list.append(p)
            if e.reason == 'NOT FOUND':  # return e if 404: page not found
                print("page not found.")
            time.sleep(1)
        except socket.timeout:  # catch timeout error
            attempts += 1
            print("\nTimeout error\n")
        except UnicodeEncodeError as e:
            with libreq.urlopen("http:" + urllib.parse.quote(url.split(':')[1])) as page:
                doc = page.read()
            success = 1
            print("Not Recognized Characters.")
        except:  # catch all rest error
            attempts += 1
            print("\nUnknown error\n")
        
    if not success:
        print("failed\n")
        continue
    
    soup = bs(doc)
    a = soup.findAll("div", {"class":'infobox-cell-2'})
    l3 = {}
    l = []
    #te = a[0].text.strip()
    for i in range(4):
        te = a[i].text.strip()
        print(te)
        l.append(te)

    for idx in range(0, len(l), 2):
        s = l[idx]
        l3[s[:-1]] = l[idx+1]
    l3['Team Name'] = t 
    list_team.append(l3)
    already_dealt.append(t.lower())

Atlanta Reign
OWL City:
Atlanta
Location:
United States
Boston Uprising
OWL City:
Boston
Location:
United States
Chengdu_Hunters
OWL City:
Chengdu
Location:
China
Dallas Fuel
OWL City:
Dallas
Location:
United States
Florida Mayhem
OWL City:
Miami-Orlando
Location:
United States
Guangzhou Charge
OWL City:
Guangzhou
Location:
China
Hangzhou Spark
OWL City:
Hangzhou
Location:
China
Houston Outlaws
OWL City:
Houston
Location:
United States
London Spitfire
OWL City:
London
Location:
United Kingdom
Los Angeles Gladiators
OWL City:
Los Angeles
Location:
United States
Los Angeles Valiant
OWL City:
Los Angeles
Location:
United States
New York Excelsior
OWL City:
New York
Location:
United States
Paris Eternal
OWL City:
Paris
Location:
France
Philadelphia Fusion
OWL City:
Philadelphia
Location:
United States
San Francisco Shock
OWL City:
San Francisco
Location:
United States
Seoul Dynasty
OWL City:
Seoul
Location:
South Korea
Shanghai Dragons
OWL City:
Shanghai
Location:
China
Toronto Defiant
OWL

## Write to csv

In [91]:
df = pd.DataFrame(data=list_player)
df.to_csv("player_info.csv")
df = pd.DataFrame(data=history_list)
df.to_csv("player_history.csv")
df = pd.DataFrame(data=list_team)
df.to_csv("team_info.csv")

## For debugging purposes